In [1]:
import sklearn as sk
import pandas as pd
import numpy as np
from os import getenv
from datetime import datetime
from amb_sdk.sdk import DarwinSdk
import matplotlib.pyplot as plt

%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Import Data Files

In [3]:
cycle_df = pd.read_csv('./data/cleaned_final_b_cycle_data.csv')
weather_df = pd.read_csv('./data/cleaned_austin_weather.csv')

In [4]:
cycle_df.head()

,Trip ID,Membership Type,Bicycle ID,Checkout Date,Checkout Time,Checkout Kiosk,Return Kiosk,Trip Duration Minutes,Month,Year,Checkout WeekDay
0,9900285854,Annual (San Antonio B-cycle),207.0,2014-10-26,0 days 13:12:00.000000000,West & 6th St.,Rainey St @ Cummings,76,10,2014,Sunday
1,9900285855,24-Hour Kiosk (Austin B-cycle),969.0,2014-10-26,0 days 13:12:00.000000000,Convention Center / 4th St. @ MetroRail,Pfluger Bridge @ W 2nd Street,58,10,2014,Sunday
2,9900285856,Annual Membership (Austin B-cycle),214.0,2014-10-26,0 days 13:12:00.000000000,West & 6th St.,8th & Congress,8,10,2014,Sunday
3,9900285857,24-Hour Kiosk (Austin B-cycle),745.0,2014-10-26,0 days 13:12:00.000000000,Zilker Park at Barton Springs & William Barton...,Zilker Park at Barton Springs & William Barton...,28,10,2014,Sunday
4,9900285858,24-Hour Kiosk (Austin B-cycle),164.0,2014-10-26,0 days 13:12:00.000000000,Bullock Museum @ Congress & MLK,Convention Center/ 3rd & Trinity,15,10,2014,Sunday


In [5]:
cycle_df.describe()

,Trip ID,Bicycle ID,Trip Duration Minutes,Month,Year
count,6.194640e+05,618807.000000,619464.000000,619464.000000,619464.000000
mean,5.366527e+09,471.195502,29.893908,5.903055,2015.420898
std,4.926918e+09,323.385229,70.892816,3.131094,1.057068
min,8.269930e+06,3.000000,3.000000,1.000000,2013.000000
25%,1.271565e+07,208.000000,9.000000,3.000000,2015.000000
50%,9.900027e+09,417.000000,15.000000,6.000000,2015.000000
75%,9.900189e+09,735.000000,29.000000,9.000000,2016.000000
max,9.900353e+09,5089.000000,10981.000000,12.000000,2017.000000


In [6]:
weather_df.head()

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,SeaLevelPressureHighInches,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,2013-12-21,74,60,45,67,49,43,93,75,57,29.86,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,2013-12-22,56,48,39,43,36,28,93,68,43,30.41,30.13,29.87,10,10,5,16,6,25,0,
2,2013-12-23,58,45,32,31,27,23,76,52,27,30.56,30.49,30.41,10,10,10,8,3,12,0,
3,2013-12-24,61,46,31,36,28,21,89,56,22,30.56,30.45,30.3,10,10,7,12,4,20,0,
4,2013-12-25,58,50,41,44,40,36,86,71,56,30.41,30.33,30.27,10,10,7,10,2,16,T,


In [7]:
weather_df.describe()

,TempHighF,TempAvgF,TempLowF
count,1319.000000,1319.000000,1319.000000
mean,80.862775,70.642911,59.902957
std,14.766523,14.045904,14.190648
min,32.000000,29.000000,19.000000
25%,72.000000,62.000000,49.000000
50%,83.000000,73.000000,63.000000
75%,92.000000,83.000000,73.000000
max,107.000000,93.000000,81.000000


# Data Cleaning

## Type of conversion

In [8]:
print('\nBefore:')
print(cycle_df[['Checkout Date', 'Checkout Time']].dtypes)

cycle_df['Checkout Date'] = pd.to_datetime(
    cycle_df['Checkout Date'], infer_datetime_format=True
    ).dt.normalize() 

cycle_df['Checkout Time'] = pd.to_timedelta(
    cycle_df['Checkout Time']) 

print('\nAfter:')
print(cycle_df[['Checkout Date', 'Checkout Time']].dtypes)


Before:
Checkout Date    object
Checkout Time    object
dtype: object

After:
Checkout Date     datetime64[ns]
Checkout Time    timedelta64[ns]
dtype: object


In [9]:
print('\nBefore:')
print(weather_df['Date'].dtypes)

weather_df['Date'] = pd.to_datetime(
    weather_df['Date'], infer_datetime_format=True
    ).dt.normalize() 

print('\nAfter:')
print(weather_df['Date'].dtypes)


Before:
object

After:
datetime64[ns]


# Feature Engineering

Creating a dataframe of total rides for each day.

In [11]:
rides_each_day = cycle_df.groupby('Checkout Date')

day_stats = pd.DataFrame(
[
    (date.date(),
     date.date().month, 
     date.date().year, 
     date.weekday(),
     round(group['Trip Duration Minutes'].mean(),3),
     len(group),
    )
    for date,group in rides_each_day
], 
    columns=['Date','Month','Year','Weekday', 
             'Avg Trip Duration',
            'Total Rides'])

day_stats['Date'] = pd.to_datetime(
    day_stats['Date'], infer_datetime_format=True
    ).dt.normalize()

day_stats.sort_values('Total Rides', ascending=False).head(10)

,Date,Month,Year,Weekday,Avg Trip Duration,Total Rides
452,2015-03-19,3,2015,3,21.077,2927
447,2015-03-14,3,2015,5,26.473,2645
83,2014-03-14,3,2014,4,37.012,2642
448,2015-03-15,3,2015,6,22.700,2539
84,2014-03-15,3,2014,5,40.202,2463
451,2015-03-18,3,2015,2,23.004,2376
449,2015-03-16,3,2015,0,19.625,2266
450,2015-03-17,3,2015,1,21.252,2134
816,2016-03-17,3,2016,3,24.781,2129
1121,2017-03-18,3,2017,5,34.351,2127


   ## Merging Cycle Data with Weather Data

In [12]:
filt_weather_df = weather_df[['Date','TempHighF','TempAvgF','DewPointHighF', 'DewPointAvgF',
                              'WindHighMPH','WindAvgMPH','PrecipitationSumInches','Events']]

merge_df = pd.merge(day_stats, filt_weather_df, on='Date')
print(merge_df.dtypes)
merge_df.head()

Date                      datetime64[ns]
Month                              int64
Year                               int64
Weekday                            int64
Avg Trip Duration                float64
Total Rides                        int64
TempHighF                          int64
TempAvgF                           int64
DewPointHighF                     object
DewPointAvgF                      object
WindHighMPH                       object
WindAvgMPH                        object
PrecipitationSumInches            object
Events                            object
dtype: object


,Date,Month,Year,Weekday,Avg Trip Duration,Total Rides,TempHighF,TempAvgF,DewPointHighF,DewPointAvgF,WindHighMPH,WindAvgMPH,PrecipitationSumInches,Events
0,2013-12-21,12,2013,5,30.887,97,74,60,67,49,20,4,0.46,"Rain , Thunderstorm"
1,2013-12-22,12,2013,6,43.385,109,56,48,43,36,16,6,0,
2,2013-12-23,12,2013,0,57.376,93,58,45,31,27,8,3,0,
3,2013-12-24,12,2013,1,33.519,81,61,46,36,28,12,4,0,
4,2013-12-25,12,2013,2,50.695,141,58,50,44,40,10,2,T,


## Processing Merged DataFrame

In [13]:
merge_df.Events.replace(' ', 'Sunny', inplace=True)


#I am making the T (trace) equal to 0.005 inches.
merge_df['PrecipitationSumInches'].replace('T',0.005, inplace=True)
# else I can remove them
# merge_df = merge_df[~merge_df['PrecipitationSumInches'].isin(['T'])]


# merge_df[~merge_df['DewPointHighF'].isin(['-'])].DewPointAvgF.astype('int').mean()
# This line above ^ returned a mean of 61.62
# merge_df[~merge_df['DewPointAvgF'].isin(['-'])].DewPointAvgF.astype('int').mean()
# This line above ^ returned a mean of 56.92
# merge_df[~merge_df['WindHighMPH'].isin(['-'])].WindHighMPH.astype('int').mean()
# This line above ^ returned a mean of 13.22
# merge_df[~merge_df['WindAvgMPH'].isin(['-'])].WindAvgMPH.astype('int').mean()
# This line above ^ returned a mean of 5.01



merge_df['DewPointHighF'].replace('-', 62, inplace=True)
merge_df['DewPointAvgF'].replace( '-', 57, inplace=True)
merge_df['WindHighMPH'].replace( '-', 13, inplace=True)
merge_df['WindAvgMPH'].replace('-', 5, inplace=True)


cols = merge_df.columns[merge_df.dtypes.eq('object')][:-1]
merge_df[cols] = merge_df[cols].apply(pd.to_numeric)
merge_df.dtypes
#merge_df.head()

Date                      datetime64[ns]
Month                              int64
Year                               int64
Weekday                            int64
Avg Trip Duration                float64
Total Rides                        int64
TempHighF                          int64
TempAvgF                           int64
DewPointHighF                      int64
DewPointAvgF                       int64
WindHighMPH                        int64
WindAvgMPH                         int64
PrecipitationSumInches           float64
Events                            object
dtype: object

### Binning Precipitation Inches

In [14]:
precip_bins = [0,0.001, 0.5, 2,10]
merge_df['PrecipBins'] = pd.cut(merge_df.PrecipitationSumInches.astype('float64'), precip_bins,
                                labels=['0','0.1-0.5','0.5-2','2-10'],
                                include_lowest=True
                                )

merge_df['PrecipBins'].unique()

[0.1-0.5, 0, 0.5-2, 2-10]
Categories (4, object): [0 < 0.1-0.5 < 0.5-2 < 2-10]

### Viewing Rides by Day of the week for different Rain Bins


In [23]:
rides_by_rain = merge_df.groupby(['PrecipBins', 'Weekday'])

day_conv_dict = {
    0:'Monday',
    1:'Tuesday',
    2:'Wednesday',
    3:'Thursday',
    4:'Friday',
    5:'Saturday',
    6:'Sunday'
}

weather_rides = []
print('Day,  Rain Bin,  Count')
for index,(name,group) in enumerate(rides_by_rain):
    print(f"{day_conv_dict[index%7]},   {name[0]},   {group['Total Rides'].sum()}")
    weather_rides.append((name[0],group['Total Rides'].sum()))
    

Day,  Rain Bin,  Count
Monday,   0,   52326
Tuesday,   0,   49813
Wednesday,   0,   46905
Thursday,   0,   48410
Friday,   0,   68001
Saturday,   0,   91228
Sunday,   0,   76639
Monday,   0.1-0.5,   17656
Tuesday,   0.1-0.5,   12625
Wednesday,   0.1-0.5,   17715
Thursday,   0.1-0.5,   20806
Friday,   0.1-0.5,   25199
Saturday,   0.1-0.5,   37051
Sunday,   0.1-0.5,   27746
Monday,   0.5-2,   3087
Tuesday,   0.5-2,   2035
Wednesday,   0.5-2,   1283
Thursday,   0.5-2,   1578
Friday,   0.5-2,   4393
Saturday,   0.5-2,   7153
Sunday,   0.5-2,   4741
Monday,   2-10,   770
Tuesday,   2-10,   314
Wednesday,   2-10,   344
Thursday,   2-10,   838
Friday,   2-10,   139
Saturday,   2-10,   94
Sunday,   2-10,   575


## Notes
Rain has a severe impact on how many bikes are checked out from B cycle.  
Even with a little bit of rain you see a sharp decrease in bike checkouts.  
However, once you go over an 1-2 inches of rain the trend associated with the day of the week goes away.  
Saturday actually becomes the least popular day if it rains over 2 inches. 



In [24]:
merge_df.head()

,Date,Month,Year,Weekday,Avg Trip Duration,Total Rides,TempHighF,TempAvgF,DewPointHighF,DewPointAvgF,WindHighMPH,WindAvgMPH,PrecipitationSumInches,Events,PrecipBins
0,2013-12-21,12,2013,5,30.887,97,74,60,67,49,20,4,0.460,"Rain , Thunderstorm",0.1-0.5
1,2013-12-22,12,2013,6,43.385,109,56,48,43,36,16,6,0.000,Sunny,0
2,2013-12-23,12,2013,0,57.376,93,58,45,31,27,8,3,0.000,Sunny,0
3,2013-12-24,12,2013,1,33.519,81,61,46,36,28,12,4,0.000,Sunny,0
4,2013-12-25,12,2013,2,50.695,141,58,50,44,40,10,2,0.005,Sunny,0.1-0.5


## Creating CSV for Darwin

In [25]:
merge_df.to_csv('./data/final_cycle_weather_by_day.csv', index=False)